<h2>定义部分：参考<a href="https://github.com/WangliLin/xunfei2021_car_loan_top1">https://github.com/WangliLin/xunfei2021_car_loan_top1</a></h2>

In [15]:
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, auc, roc_curve, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from scipy.stats import variation
from tqdm import tqdm
import pickle
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
from sklearn.preprocessing import StandardScaler, QuantileTransformer, KBinsDiscretizer, LabelEncoder, MinMaxScaler, PowerTransformer
from sklearn.preprocessing import OrdinalEncoder

def train_lgb_kfold(X_train, y_train, X_test, n_fold=5, cate_feats=None):
    '''train lightgbm with k-fold split'''
    gbms = []
    kfold = StratifiedKFold(n_splits=n_fold, random_state=1024, shuffle=True)
    oof_preds = np.zeros((X_train.shape[0],))
    test_preds = np.zeros((X_test.shape[0],))

    for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
        logging.info(f'############ fold {fold} ###########')
        X_tr, X_val, y_tr, y_val = X_train.iloc[train_index], X_train.iloc[val_index], y_train[train_index], y_train[val_index]
        dtrain = lgb.Dataset(X_tr, y_tr, categorical_feature=cate_feats)
        dvalid = lgb.Dataset(X_val, y_val, categorical_feature=cate_feats, reference=dtrain)

        params = {
            'objective': 'binary',
            'metric': 'auc',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'min_data_in_leaf': 50,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'n_jobs': -1,
            'seed': 1024
        }

        gbm = lgb.train(params,
                        dtrain,
                        num_boost_round=300,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=50,
                        early_stopping_rounds=20)

        oof_preds[val_index] = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        test_preds += gbm.predict(X_test, num_iteration=gbm.best_iteration) / kfold.n_splits
        gbms.append(gbm)

    return gbms, oof_preds, test_preds

<h2>读取数据部分</h2>

In [16]:
data = pd.read_csv('../data/账户交易信息.csv')
data_static = pd.read_csv('../data/账户静态信息.csv')
data_label = pd.read_csv('../data/训练集标签.csv')

<h2>特征工程部分</h2>

In [17]:
d = {}
users = data.zhdh.unique().tolist()
for user_id in tqdm(users):
    data_sample = data[data.zhdh == user_id].reset_index(drop=True)
    data_sample["转账日期"] = data_sample["jyrq"] + " " + data_sample["jysj"]
    data_sample["转账日期"] = pd.to_datetime(data_sample["转账日期"])
    data_sample["年月"] = data_sample["转账日期"].apply(lambda x: x.year * 10000 + x.month)
    data_sample["对方账户"] = data_sample["dfzh"] + data_sample["dfhh"]
    
    # 最大进账频次，最小进账频次，最大出账频次，最小出账频次
    max_in_cnt = data_sample[data_sample["jdbj"] == 1]["jyrq"].value_counts().max()
    min_in_cnt = data_sample[data_sample["jdbj"] == 1]["jyrq"].value_counts().min()
    max_out_cnt = data_sample[data_sample["jdbj"] == 0]["jyrq"].value_counts().max()
    min_out_cnt = data_sample[data_sample["jdbj"] == 0]["jyrq"].value_counts().min()
    
    # 对方账户的出现频次的最大值、最小值、中位数
    max_df_cnt = data_sample["对方账户"].value_counts().max()
    min_df_cnt = data_sample["对方账户"].value_counts().min()
    median_df_cnt = data_sample["对方账户"].value_counts().median()
    
    # 转入金额、转出金额的变异系数
    in_amt_variation = variation(data_sample[data_sample.jdbj == 1].jyje.values) 
    out_amt_variation = variation(data_sample[data_sample.jdbj == 0].jyje)
    
    all_cnt = data_sample.shape[0] # 操作次数
    
    # 转入次数、转出次数、转入次数占比
    in_cnt = data_sample.jdbj.sum()
    out_cnt = all_cnt - in_cnt
    in_ratio = in_cnt / all_cnt
    
    # 转入金额、转出金额、转入金额占比
    in_amt = data_sample[data_sample.jdbj == 1].jyje.sum()
    out_amt = data_sample[data_sample.jdbj == 0].jyje.sum()
    in_amt_ratio = in_amt / (in_amt + out_amt)
    
    # 转入人数、转出人数、转入人数占比
    in_user_cnt = data_sample[data_sample.jdbj == 1].dfzh.nunique()
    out_user_cnt = data_sample[data_sample.jdbj == 0].dfzh.nunique()
    in_user_ratio = in_user_cnt / (in_user_cnt + out_user_cnt)
    
    date_cnt = data_sample.jyrq.nunique() # 转账日期数
    
    d[user_id] = [all_cnt, in_cnt, out_cnt, in_ratio, in_amt, out_amt, in_amt_ratio, in_amt_variation, out_amt_variation, in_user_cnt, out_user_cnt, in_user_ratio, date_cnt, max_in_cnt, min_in_cnt, max_out_cnt, min_out_cnt, max_in_ratio_month, min_in_ratio_month, median_in_ratio_month, max_df_cnt, min_df_cnt, median_df_cnt]

data_df = pd.DataFrame.from_dict(d).T.reset_index()
data_df.columns = ['zhdh', 'all_cnt', 'in_cnt', 'out_cnt', 'in_ratio', 'in_amt', 'out_amt', 'in_amt_ratio', "in_amt_variation", "out_amt_variation", 'in_user_cnt', 'out_user_cnt', 'in_user_ratio', 'date_cnt', "max_in_cnt", "min_in_cnt", "max_out_cnt", "min_out_cnt", "max_in_ratio_month", "min_in_ratio_month", "median_in_ratio_month", "max_df_cnt", "min_df_cnt", "median_df_cnt"]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 6000/6000 [03:27<00:00, 28.92it/s]


<h2>合并标签部分</h2>

In [18]:
data_static.columns = ['zhdh', 'khrq', 'khjgdh', 'xb', 'age']
df_feats = pd.merge(left=data_df, right=data_static[['zhdh', 'xb', 'age']], on='zhdh', how='left')
df_final = pd.merge(left=df_feats, right=data_label, on='zhdh', how='left')

<h2>准备数据集</h2>

In [19]:
df_train = df_final[df_final.black_flag.notnull()].reset_index(drop=True)
df_test = df_final[df_final.black_flag.isnull()].reset_index(drop=True)

feats = df_train.columns[1:-1].tolist()
X_train = df_train[feats]
y_train = df_train['black_flag']
X_test = df_test[feats]

<h2>使用SMOTE增强</h2>

In [20]:
# smo = SMOTE(random_state=42)
# X_train, y_train = smo.fit_sample(X_train, y_train)

<h2>开始训练</h2>

In [21]:
gbms, oof_preds, test_preds = train_lgb_kfold(X_train, y_train, X_test, n_fold=5)

2023-02-14 14:30:14,843 : INFO : ############ fold 0 ###########
2023-02-14 14:30:14,909 : INFO : ############ fold 1 ###########
2023-02-14 14:30:14,965 : INFO : ############ fold 2 ###########
2023-02-14 14:30:15,005 : INFO : ############ fold 3 ###########


Training until validation scores don't improve for 20 rounds
[50]	training's auc: 0.987558	valid_1's auc: 0.9675
Early stopping, best iteration is:
[57]	training's auc: 0.989583	valid_1's auc: 0.969259
Training until validation scores don't improve for 20 rounds
[50]	training's auc: 0.987303	valid_1's auc: 0.947407
Early stopping, best iteration is:
[62]	training's auc: 0.990856	valid_1's auc: 0.94963
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[29]	training's auc: 0.980041	valid_1's auc: 0.9175
Training until validation scores don't improve for 20 rounds
[50]	training's auc: 0.990064	valid_1's auc: 0.940648


2023-02-14 14:30:15,067 : INFO : ############ fold 4 ###########


Early stopping, best iteration is:
[64]	training's auc: 0.992969	valid_1's auc: 0.943704
Training until validation scores don't improve for 20 rounds
[50]	training's auc: 0.986968	valid_1's auc: 0.943796
Early stopping, best iteration is:
[73]	training's auc: 0.992749	valid_1's auc: 0.94963


<h2>搜索最优阈值</h2>

In [22]:
def gen_thres_new(df_train, oof_preds):
    df_train['oof_preds'] = oof_preds
    quantile_point = df_train['black_flag'].mean()
    thres = df_train['oof_preds'].quantile(1 - quantile_point)

    _thresh = []
    for thres_item in np.arange(thres - 0.2, thres + 0.2, 0.01):
        _thresh.append(
            [thres_item, f1_score(df_train['black_flag'], np.where(oof_preds > thres_item, 1, 0), average='macro')])

    _thresh = np.array(_thresh)
    best_id = _thresh[:, 1].argmax()
    best_thresh = _thresh[best_id][0]

    print("阈值: {}\n训练集的f1: {}".format(best_thresh, _thresh[best_id][1]))
    return best_thresh

best_thresh = gen_thres_new(df_train, oof_preds)

阈值: 0.349858525945368
训练集的f1: 0.8883701001004481


<h2>生成提交结果</h2>

In [23]:
df_test['black_flag'] = np.where(test_preds > best_thresh, 1, 0)
df_test[['zhdh', 'black_flag']].to_csv("../data/submit_%s.csv" % datetime.now().strftime("%Y-%m-%dT%H-%M"))